<a href="https://colab.research.google.com/github/jjandnn/fsgan/blob/master/FSGAN%EF%BC%88conda%E7%89%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FSGAN**
作者：[Yuval Nirkin](https://github.com/YuvalNirkin)

colab： jjandnn ， zhuhaozh ， wangchao

# 安装

In [0]:
# 必须 p100 
!nvidia-smi

In [0]:
!rm -rf /content/sample_data
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

In [0]:
# 依赖，请勿修改安装顺序
!sudo apt-get install build-essential cmake unzip pkg-config
!sudo apt-get install libjpeg-dev libpng-dev 
!sudo apt-get install libavcodec-dev libavformat-dev libswscale-dev libv4l-dev 
!sudo apt-get install libxvidcore-dev libx264-dev
!sudo apt-get install libatlas-base-dev gfortran
!sudo apt-get install python3-dev
!conda install numpy -y

!sudo add-apt-repository ppa:jonathonf/ffmpeg-4
!sudo apt-get update
!sudo apt-get install ffmpeg

!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch -y
!conda install -c anaconda ipykernel -y
!conda install -c conda-forge yacs -y
!pip install --upgrade tensorflow
!pip install tensorboardX
!pip install ffmpeg-python
!pip install face_alignment

In [0]:
# 1，云盘源码安装（首次需要，之后使用无需再装）

%cd /content/drive/My Drive
!mkdir -p fsganLab/fshome
%cd /content/drive/My Drive/fsganLab/fshome

!git clone https://github.com/YuvalNirkin/fsgan.git
!git clone https://github.com/YuvalNirkin/face_detection_dsfd

In [0]:
# 问官方要的下载文件授权，放在fshome下，运行得到权重文件夹 weights，里面共17个模型文件
# 问官方要的下载文件授权，放在fshome下，运行得到权重文件夹 weights，里面共17个模型文件
# 问官方要的下载文件授权，放在fshome下，运行得到权重文件夹 weights，里面共17个模型文件


!python download_fsgan_models.py -m v1
!python download_fsgan_models.py -m v2
# 由于 colab 空格问题，重新组织下目录结构
!mv /content/drive/My\ Drive/fsganLab/fshome/weights /content/drive/My\ Drive/fsganLab
!cp /content/drive/My\ Drive/fsganLab/fshome/fsgan/inference/swap.py .

In [0]:
# 注册 fshome
!echo 'export PYTHONPATH=$PYTHONPATH:/content/drive/My\ Drive/fsganLab/fshome' >> ~/.profile

In [0]:
!source ~/.profile

In [0]:
# # 编译安装 opencv (如果要 avc1 编码则需要，无比费时，容易出错)
# !wget -O opencv.zip https://github.com/opencv/opencv/archive/4.3.0.zip
# !unzip opencv.zip
# !rm opencv.zip
# %cd /content/opencv-4.3.0
# !mkdir build
# %cd /content/opencv-4.3.0/build
# !cmake -D CMAKE_BUILD_TYPE=RELEASE \
# -D CMAKE_INSTALL_PREFIX=$(python -c "import sys; print(sys.prefix)") \
# -D ENABLE_FAST_MATH=ON \
# -D INSTALL_PYTHON_EXAMPLES=OFF \
# -D INSTALL_C_EXAMPLES=OFF \
# -D OPENCV_ENABLE_NONFREE=OFF \
# -D CMAKE_INSTALL_PREFIX=$(python -c "import sys; print(sys.prefix)") \
# -D PYTHON_EXECUTABLE=$(which python) \
# -D PYTHON_INCLUDE_DIR=$(python -c "from distutils.sysconfig import get_python_inc; print(get_python_inc())") \
# -D PYTHON_PACKAGES_PATH=$(python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())") \
# -D BUILD_EXAMPLES=OFF \
# -D BUILD_JPEG=ON ..
# !make -j8
# !sudo make install
# !sudo ldconfig

# 修改三个文件，防止因为没有编码器报错（你只需要按就 ok）

In [0]:
%cd /content/drive/My Drive/fsganLab/fshome

In [0]:
#@title 1，使用 mp4v 需要修改 fsgan/utils/video_renderer.py
%%writefile fsgan/utils/video_renderer.py
import numpy as np
import cv2
import torch
import torch.multiprocessing as mp
from fsgan.utils.img_utils import tensor2bgr
from fsgan.utils.bbox_utils import crop2img, scale_bbox


class VideoRenderer(mp.Process):
    """ Renders input video frames to both screen and video file.

    For more control on the rendering, this class should be inherited from and the on_render method overridden
    with an application specific implementation.

    Args:
        display (bool): If True, the rendered video will be displayed on screen
        verbose (int): Verbose level. Controls the amount of debug information in the rendering
        verbose_size (tuple of int): The rendered frame size for verbose level other than zero (width, height)
        output_crop (bool): If True, a cropped frame of size (resolution, resolution) will be rendered for
            verbose level zero
        resolution (int): Determines the size of cropped frames to be (resolution, resolution)
        crop_scale (float): Multiplier factor to scale tight bounding boxes
    """
    def __init__(self, display=False, verbose=0, verbose_size=None, output_crop=False, resolution=256, crop_scale=1.2):
        super(VideoRenderer, self).__init__()
        self._display = display
        self._verbose = verbose
        self._verbose_size = verbose_size
        self._output_crop = output_crop
        self._resolution = resolution
        self._crop_scale = crop_scale
        self._running = True
        self._input_queue = mp.Queue()
        self._reply_queue = mp.Queue()
        self._fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self._in_vid = None
        self._out_vid = None
        self._seq = None
        self._in_vid_path = None
        self._total_frames = None
        self._frame_count = 0

    def init(self, in_vid_path, seq, out_vid_path=None, **kwargs):
        """ Initialize the video render for a new video rendering job.

        Args:
            in_vid_path (str): Input video path
            seq (Sequence): Input sequence corresponding to the input video
            out_vid_path (str, optional): If specified, the rendering will be written to an output video in that path
            **kwargs (dict): Additional keyword arguments that will be added as members of the class. This allows
                inheriting classes to access those arguments from the new process
        """
        self._input_queue.put([in_vid_path, seq, out_vid_path, kwargs])

    def write(self, *args):
        """ Add tensors for rendering.

        Args:
            *args (tuple of torch.Tensor): The tensors for rendering
        """
        self._input_queue.put([a.cpu() for a in args])

    def wait_until_finished(self):
        """ Wait for the video renderer to finish the current video rendering job. """
        return self._reply_queue.get()

    def on_render(self, *args):
        """ Given the input tensors this method produces a cropped rendered image.

        This method should be overridden by inheriting classes to customize the rendering. By default this method
        expects the first tensor to be a cropped image tensor of shape (B, 3, H, W) where B is the batch size,
        H is the height of the image and W is the width of the image.

        Args:
            *args (tuple of torch.Tensor): The tensors for rendering

        Returns:
            render_bgr (np.array): The cropped rendered image
        """
        return tensor2bgr(args[0])

    def run(self):
        """ Main processing loop. Intended to be executed on a separate process. """
        while self._running:
            task = self._input_queue.get()

            # Initialize new video rendering task
            if self._in_vid is None:
                self._in_vid_path, self._seq, out_vid_path = task[:3]
                additional_attributes = task[3]
                self._frame_count = 0

                # Add additional arguments as members
                for attr_name, attr_val in additional_attributes.items():
                    setattr(self, attr_name, attr_val)

                # Open input video
                self._in_vid = cv2.VideoCapture(self._in_vid_path)
                assert self._in_vid.isOpened(), f'Failed to open video: "{self._in_vid_path}"'

                in_total_frames = int(self._in_vid.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = self._in_vid.get(cv2.CAP_PROP_FPS)
                in_vid_width = int(self._in_vid.get(cv2.CAP_PROP_FRAME_WIDTH))
                in_vid_height = int(self._in_vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self._total_frames = in_total_frames if self._verbose == 0 else len(self._seq)
                # print(f'Debug: initializing video: "{self._in_vid_path}", total_frames={self._total_frames}')

                # Initialize output video
                if out_vid_path is not None:
                    out_size = (in_vid_width, in_vid_height)
                    if self._verbose <= 0 and self._output_crop:
                        out_size = (self._resolution, self._resolution)
                    elif self._verbose_size is not None:
                        out_size = self._verbose_size
                    self._out_vid = cv2.VideoWriter(out_vid_path, self._fourcc, fps, out_size)

                # Write frames as they are until the start of the sequence
                if self._verbose == 0:
                    for i in range(self._seq.start_index):
                        # Read frame
                        ret, frame_bgr = self._in_vid.read()
                        assert frame_bgr is not None, f'Failed to read frame {i} from input video: "{self._in_vid_path}"'
                        self._render(frame_bgr)
                        self._frame_count += 1

                continue

            # Write a batch of frames
            tensors = task
            batch_size = tensors[0].shape[0]

            # For each frame in the current batch of tensors
            for b in range(batch_size):
                # Handle full frames if output_crop was not specified
                full_frame_bgr, bbox = None, None
                if self._verbose == 0 and not self._output_crop:
                    # Read frame from input video
                    ret, full_frame_bgr = self._in_vid.read()
                    assert full_frame_bgr is not None, \
                        f'Failed to read frame {i} from input video: "{self._in_vid_path}"'

                    # Get bounding box from sequence
                    det = self._seq[self._frame_count - self._seq.start_index]
                    bbox = np.concatenate((det[:2], det[2:] - det[:2]))
                    bbox = scale_bbox(bbox, self._crop_scale)

                render_bgr = self.on_render(*[t[b] for t in tensors])
                self._render(render_bgr, full_frame_bgr, bbox)
                self._frame_count += 1
                # print(f'Debug: Writing frame: {self._frame_count}')

            # Check if we reached the end of the sequence
            if self._verbose == 0 and self._frame_count >= (self._seq.start_index + len(self._seq)):
                for i in range(self._seq.start_index + len(self._seq), self._total_frames):
                    # Read frame
                    ret, frame_bgr = self._in_vid.read()
                    assert frame_bgr is not None, f'Failed to read frame {i} from input video: "{self._in_vid_path}"'
                    self._render(frame_bgr)
                    self._frame_count += 1

            # Check if all frames have been processed
            if self._frame_count >= self._total_frames:
                # Clean up
                self._in_vid.release()
                self._out_vid.release()
                self._in_vid = None
                self._out_vid = None
                self._seq = None
                self._in_vid_path = None
                self._total_frames = None
                self._frame_count = 0

                # Notify job is finished
                self._reply_queue.put(True)

    def _render(self, render_bgr, full_frame_bgr=None, bbox=None):
        if self._verbose == 0 and not self._output_crop and full_frame_bgr is not None:
            render_bgr = crop2img(full_frame_bgr, render_bgr, bbox)
        if self._out_vid is not None:
            self._out_vid.write(render_bgr)
        if self._display:
            cv2.imshow('render', render_bgr)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self._running = False


In [0]:
#@title 2，使用 mp4v 需要修改 fsgan/preprocess/preprocess_video.py
%%writefile fsgan/preprocess/preprocess_video.py
""" Video preprocessing.

This script implements all preprocessing required for both training and inference.
The preprocessing information will be cached in a directory by the file's name without the extension,
residing in the same directory as the file. The information contains: face detections, face sequences,
and cropped videos per sequence. In addition for each cropped video, the corresponding pose, landmarks, and
segmentation masks will be computed and cached.
"""

import os
import argparse
import pickle
from tqdm import tqdm
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from face_detection_dsfd.face_detector import FaceDetector
from fsgan.utils.utils import set_device, load_model
from fsgan.preprocess.detections2sequences_center import main as detections2sequences_main
from fsgan.preprocess.crop_video_sequences import main as crop_video_sequences_main
from fsgan.preprocess.crop_image_sequences import main as crop_image_sequences_main
from fsgan.datasets.video_inference_dataset import VideoInferenceDataset
import fsgan.datasets.img_landmarks_transforms as img_landmarks_transforms
from fsgan.datasets.img_landmarks_transforms import Resize, ToTensor
from fsgan.utils.temporal_smoothing import TemporalSmoothing
from fsgan.utils.landmarks_utils import LandmarksHeatMapEncoder, smooth_landmarks_98pts
from fsgan.utils.seg_utils import encode_binary_mask, remove_inner_mouth
from fsgan.utils.batch import main as batch


base_parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter, add_help=False)

general = base_parser.add_argument_group('general')
general.add_argument('-r', '--resolution', default=256, type=int, metavar='N',
                     help='finest processing resolution')
general.add_argument('-cs', '--crop_scale', default=1.2, type=float, metavar='F',
                     help='crop scale relative to bounding box')
general.add_argument('--gpus', default=None, nargs='+', type=int, metavar='N',
                     help='list of gpu ids to use')
general.add_argument('--cpu_only', action='store_true',
                     help='force cpu only')
general.add_argument('-d', '--display', action='store_true',
                     help='display the rendering')
general.add_argument('-v', '--verbose', default=0, type=int, metavar='N',
                     help='verbose level')

detection = base_parser.add_argument_group('detection')
detection.add_argument('-dm', '--detection_model', metavar='PATH', default='../weights/WIDERFace_DSFD_RES152.pth',
                       help='path to face detection model')
detection.add_argument('-db', '--det_batch_size', default=8, type=int, metavar='N',
                       help='detection batch size')
detection.add_argument('-dp', '--det_postfix', default='_dsfd.pkl', metavar='POSTFIX',
                       help='detection file postfix')

sequences = base_parser.add_argument_group('sequences')
sequences.add_argument('-it', '--iou_thresh', default=0.75, type=float,
                       metavar='F', help='IOU threshold')
sequences.add_argument('-ml', '--min_length', default=10, type=int,
                       metavar='N', help='minimum sequence length')
sequences.add_argument('-ms', '--min_size', default=64, type=int,
                       metavar='N', help='minimum sequence average bounding box size')
sequences.add_argument('-ck', '--center_kernel', default=25, type=int,
                       metavar='N', help='center average kernel size')
sequences.add_argument('-sk', '--size_kernel', default=51, type=int,
                       metavar='N', help='size average kernel size')
sequences.add_argument('-dsd', '--disable_smooth_det', dest='smooth_det', action='store_false',
                       help='disable smoothing the detection bounding boxes')
sequences.add_argument('-sp', '--seq_postfix', default='_dsfd_seq.pkl', metavar='POSTFIX',
                       help='sequence file postfix')
sequences.add_argument('-we', '--write_empty', action='store_true',
                       help='write empty sequence lists to file')

pose = base_parser.add_argument_group('pose')
pose.add_argument('-pm', '--pose_model', default='../weights/hopenet_robust_alpha1.pth', metavar='PATH',
                       help='path to face pose model file')
pose.add_argument('-pb', '--pose_batch_size', default=128, type=int, metavar='N',
                       help='pose batch size')
pose.add_argument('-pp', '--pose_postfix', default='_pose.npz', metavar='POSTFIX',
                       help='pose file postfix')
pose.add_argument('-cp', '--cache_pose', action='store_true',
                  help='Toggle whether to cache pose')
pose.add_argument('-cf', '--cache_frontal', action='store_true',
                  help='Toggle whether to cache frontal images for each sequence')
pose.add_argument('-spo', '--smooth_poses', default=5, type=int, metavar='N',
                  help='poses temporal smoothing kernel size')

landmarks = base_parser.add_argument_group('landmarks')
landmarks.add_argument('-lm', '--lms_model', default='../weights/hr18_wflw_landmarks.pth', metavar='PATH',
                       help='landmarks model')
landmarks.add_argument('-lb', '--lms_batch_size', default=64, type=int, metavar='N',
                       help='landmarks batch size')
landmarks.add_argument('-lp', '--landmarks_postfix', default='_lms.npz', metavar='POSTFIX',
                       help='landmarks file postfix')
landmarks.add_argument('-cl', '--cache_landmarks', action='store_true',
                       help='Toggle whether to cache landmarks')
landmarks.add_argument('-sl', '--smooth_landmarks', default=7, type=int, metavar='N',
                       help='landmarks temporal smoothing kernel size')

segmentation = base_parser.add_argument_group('segmentation')
segmentation.add_argument('-sm', '--seg_model', default='../weights/celeba_unet_256_1_2_segmentation_v2.pth',
                          metavar='PATH', help='segmentation model')
segmentation.add_argument('-sb', '--seg_batch_size', default=32, type=int, metavar='N',
                          help='segmentation batch size')
segmentation.add_argument('-sep', '--segmentation_postfix', default='_seg.pkl', metavar='POSTFIX',
                          help='segmentation file postfix')
segmentation.add_argument('-cse', '--cache_segmentation', action='store_true',
                          help='Toggle whether to cache segmentation')
segmentation.add_argument('-sse', '--smooth_segmentation', default=5, type=int, metavar='N',
                          help='segmentation temporal smoothing kernel size')
segmentation.add_argument('-srm', '--seg_remove_mouth', action='store_true',
                          help='if true, the inner part of the mouth will be removed from the segmentation')

parser = argparse.ArgumentParser(description=__doc__, formatter_class=argparse.ArgumentDefaultsHelpFormatter,
                                 parents=[base_parser])
parser.add_argument('input', metavar='VIDEO', nargs='+',
                    help='path to input video')
parser.add_argument('-o', '--output', metavar='DIR',
                    help='output directory')
d = parser.get_default


class VideoProcessBase(object):
    def __init__(self, resolution=d('resolution'), crop_scale=d('crop_scale'), gpus=d('gpus'),
         cpu_only=d('cpu_only'), display=d('display'), verbose=d('verbose'),
         # Detection arguments:
         detection_model=d('detection_model'), det_batch_size=d('det_batch_size'), det_postfix=d('det_postfix'),
         # Sequence arguments:
         iou_thresh=d('iou_thresh'), min_length=d('min_length'), min_size=d('min_size'),
         center_kernel=d('center_kernel'), size_kernel=d('size_kernel'), smooth_det=d('smooth_det'),
         seq_postfix=d('seq_postfix'), write_empty=d('write_empty'),
         # Pose arguments:
         pose_model=d('pose_model'), pose_batch_size=d('pose_batch_size'), pose_postfix=d('pose_postfix'),
         cache_pose=d('cache_pose'), cache_frontal=d('cache_frontal'), smooth_poses=d('smooth_poses'),
         # Landmarks arguments:
         lms_model=d('lms_model'), lms_batch_size=d('lms_batch_size'), landmarks_postfix=d('landmarks_postfix'),
         cache_landmarks=d('cache_landmarks'), smooth_landmarks=d('smooth_landmarks'),
         # Segmentation arguments:
         seg_model=d('seg_model'), seg_batch_size=d('seg_batch_size'), segmentation_postfix=d('segmentation_postfix'),
         cache_segmentation=d('cache_segmentation'), smooth_segmentation=d('smooth_segmentation'),
         seg_remove_mouth=d('seg_remove_mouth')):
        # General
        self.resolution = resolution
        self.crop_scale = crop_scale
        self.display = display
        self.verbose = verbose

        # Detection
        self.face_detector = FaceDetector(det_postfix, detection_model, gpus, det_batch_size, display)
        self.det_postfix = det_postfix

        # Sequences
        self.iou_thresh = iou_thresh
        self.min_length = min_length
        self.min_size = min_size
        self.center_kernel = center_kernel
        self.size_kernel = size_kernel
        self.smooth_det = smooth_det
        self.seq_postfix = seq_postfix
        self.write_empty = write_empty

        # Pose
        self.pose_batch_size = pose_batch_size
        self.pose_postfix = pose_postfix
        self.cache_pose = cache_pose
        self.cache_frontal = cache_frontal
        self.smooth_poses = smooth_poses

        # Landmarks
        self.smooth_landmarks = smooth_landmarks
        self.landmarks_postfix = landmarks_postfix
        self.cache_landmarks = cache_landmarks
        self.lms_batch_size = lms_batch_size

        # Segmentation
        self.smooth_segmentation = smooth_segmentation
        self.segmentation_postfix = segmentation_postfix
        self.cache_segmentation = cache_segmentation
        self.seg_batch_size = seg_batch_size
        self.seg_remove_mouth = seg_remove_mouth and cache_landmarks

        # Initialize device
        torch.set_grad_enabled(False)
        self.device, self.gpus = set_device(gpus, not cpu_only)

        # Load models
        self.face_pose = load_model(pose_model, 'face pose', self.device) if cache_pose else None
        self.L = load_model(lms_model, 'face landmarks', self.device) if cache_landmarks else None
        self.S = load_model(seg_model, 'face segmentation', self.device) if cache_segmentation else None

        # Initialize heatmap encoder
        self.heatmap_encoder = LandmarksHeatMapEncoder().to(self.device)

        # Initialize normalization tensors
        # Note: this is necessary because of the landmarks model
        self.img_mean = torch.as_tensor([0.5, 0.5, 0.5], device=self.device).view(1, 3, 1, 1)
        self.img_std = torch.as_tensor([0.5, 0.5, 0.5], device=self.device).view(1, 3, 1, 1)
        self.context_mean = torch.as_tensor([0.485, 0.456, 0.406], device=self.device).view(1, 3, 1, 1)
        self.context_std = torch.as_tensor([0.229, 0.224, 0.225], device=self.device).view(1, 3, 1, 1)

        # Support multiple GPUs
        if self.gpus and len(self.gpus) > 1:
            self.face_pose = nn.DataParallel(self.face_pose, self.gpus) if self.face_pose is not None else None
            self.L = nn.DataParallel(self.L, self.gpus) if self.L is not None else None
            self.S = nn.DataParallel(self.S, self.gpus) if self.S is not None else None

        # Initialize temportal smoothing
        if smooth_segmentation > 0:
            self.smooth_seg = TemporalSmoothing(3, smooth_segmentation).to(self.device)
        else:
            self.smooth_seg = None

        # Initialize output videos format
        self.fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    def process_pose(self, input_path, output_dir, seq_file_path):
        if not self.cache_pose:
            return
        input_path_no_ext, input_ext = os.path.splitext(input_path)

        # Load sequences from file
        with open(seq_file_path, "rb") as fp:  # Unpickling
            seq_list = pickle.load(fp)

        # Initialize transforms
        img_transforms = img_landmarks_transforms.Compose([
            Resize(224), ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

        # For each sequence
        for seq in seq_list:
            curr_vid_name = os.path.basename(input_path_no_ext) + '_seq%02d%s' % (seq.id, input_ext)
            curr_vid_path = os.path.join(output_dir, curr_vid_name)
            curr_pose_path = os.path.splitext(curr_vid_path)[0] + self.pose_postfix

            if os.path.isfile(curr_pose_path):
                continue
            print('=> Computing face poses for video: "%s"...' % curr_vid_name)

            # Initialize input video
            in_vid = VideoInferenceDataset(curr_vid_path, transform=img_transforms)
            in_vid_loader = DataLoader(in_vid, batch_size=self.pose_batch_size, num_workers=1, pin_memory=True,
                                       drop_last=False, shuffle=False)

            # For each batch of frames in the input video
            seq_poses = []
            for i, frame in enumerate(tqdm(in_vid_loader, unit='batches')):
                frame = frame.to(self.device)
                poses = self.face_pose(frame).div_(99.)  # Yaw, Pitch, Roll
                seq_poses.append(poses.cpu().numpy())
            seq_poses = np.concatenate(seq_poses)

            # Save landmarks to file
            seq_landmarks_smoothed = smooth_poses(seq_poses, self.smooth_poses)
            np.savez_compressed(curr_pose_path, poses=seq_poses, poses_smoothed=seq_landmarks_smoothed)

    def extract_frontal_images(self, input_path, output_dir, seq_file_path, out_postfix='.jpg', resolution=None):
        if not self.cache_frontal:
            return

        # Load sequences from file
        with open(seq_file_path, "rb") as fp:  # Unpickling
            seq_list = pickle.load(fp)

        # For each sequence
        for seq in seq_list:
            curr_vid_name = os.path.splitext(os.path.basename(input_path))[0] + '_seq%02d.mp4' % seq.id
            curr_vid_path = os.path.join(output_dir, curr_vid_name)
            curr_pose_path = os.path.splitext(curr_vid_path)[0] + self.pose_postfix
            curr_frontal_path = os.path.splitext(curr_vid_path)[0] + out_postfix

            if os.path.isfile(curr_frontal_path):
                continue

            # Open current video file
            vid = cv2.VideoCapture(curr_vid_path)
            if not vid.isOpened():
                raise RuntimeError('Failed to read video: ' + curr_vid_path)

            # Load current sequence poses
            curr_poses = np.load(curr_pose_path)['poses_smoothed']

            # Read frontal image from video
            frontal_index = np.argmin(np.linalg.norm(curr_poses, axis=1))
            vid.set(cv2.CAP_PROP_POS_FRAMES, frontal_index)
            ret, frontal_bgr = vid.read()

            # Resize image
            if resolution is not None:
                frontal_bgr = cv2.resize(frontal_bgr, (resolution, resolution), interpolation=cv2.INTER_CUBIC)

            # Write frontal image to file
            cv2.imwrite(curr_frontal_path, frontal_bgr)

    def process_landmarks(self, input_path, output_dir, seq_file_path):
        if not self.cache_landmarks:
            return
        input_path_no_ext, input_ext = os.path.splitext(input_path)

        # Load sequences from file
        with open(seq_file_path, "rb") as fp:  # Unpickling
            seq_list = pickle.load(fp)

        # Initialize transforms
        img_transforms = img_landmarks_transforms.Compose([
            ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        # For each sequence
        for seq in seq_list:
            curr_vid_name = os.path.basename(input_path_no_ext) + '_seq%02d%s' % (seq.id, input_ext)
            curr_vid_path = os.path.join(output_dir, curr_vid_name)
            curr_lms_path = os.path.splitext(curr_vid_path)[0] + self.landmarks_postfix

            if os.path.isfile(curr_lms_path):
                continue
            print('=> Computing face landmarks for video: "%s"...' % curr_vid_name)

            # Initialize input video
            in_vid = VideoInferenceDataset(curr_vid_path, transform=img_transforms)
            in_vid_loader = DataLoader(in_vid, batch_size=self.lms_batch_size, num_workers=1, pin_memory=True,
                                       drop_last=False, shuffle=False)

            # For each batch of frames in the input video
            seq_landmarks = []
            for i, frame in enumerate(tqdm(in_vid_loader, unit='batches')):
                frame = frame.to(self.device)
                H = self.L(frame)
                landmarks = self.heatmap_encoder(H)
                seq_landmarks.append(landmarks.cpu().numpy())
            seq_landmarks = np.concatenate(seq_landmarks)

            # Save landmarks to file
            seq_landmarks_smoothed = smooth_landmarks_98pts(seq_landmarks, self.smooth_landmarks)
            np.savez_compressed(curr_lms_path, landmarks=seq_landmarks, landmarks_smoothed=seq_landmarks_smoothed)

    def process_segmentation(self, input_path, output_dir, seq_file_path):
        if not self.cache_segmentation:
            return
        input_path_no_ext, input_ext = os.path.splitext(input_path)

        # Load sequences from file
        with open(seq_file_path, "rb") as fp:  # Unpickling
            seq_list = pickle.load(fp)

        # Initialize transforms
        img_transforms = img_landmarks_transforms.Compose([
            ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

        # For each sequence
        for seq in seq_list:
            curr_vid_name = os.path.basename(input_path_no_ext) + '_seq%02d%s' % (seq.id, input_ext)
            curr_vid_path = os.path.join(output_dir, curr_vid_name)
            curr_seg_path = os.path.splitext(curr_vid_path)[0] + self.segmentation_postfix

            if self.seg_remove_mouth:
                curr_lms_path = os.path.splitext(curr_vid_path)[0] + self.landmarks_postfix
                landmarks = np.load(curr_lms_path)['landmarks_smoothed']
                frame_count = 0

            if os.path.isfile(curr_seg_path):
                continue
            print('=> Computing face segmentation for video: "%s"...' % curr_vid_name)

            # Initialize input video
            in_vid = VideoInferenceDataset(curr_vid_path, transform=img_transforms)
            in_vid_loader = DataLoader(in_vid, batch_size=self.seg_batch_size, num_workers=1, pin_memory=True,
                                       drop_last=False, shuffle=False)

            # For each batch of frames in the input video
            pbar = tqdm(in_vid_loader, unit='batches')
            prev_segmentation = None
            r = self.smooth_seg.kernel_radius
            encoded_segmentations = []
            pad_prev, pad_next = r, r   # This initialization is only relevant if there is a leftover from last batch
            for i, frame in enumerate(pbar):
                frame = frame.to(self.device)

                # Compute segmentation
                raw_segmentation = self.S(frame)
                segmentation = torch.cat((prev_segmentation, raw_segmentation), dim=0) \
                    if prev_segmentation is not None else raw_segmentation
                if segmentation.shape[0] > r:
                    pad_prev, pad_next = r if prev_segmentation is None else 0, min(r, self.seg_batch_size - frame.shape[0])
                    segmentation = self.smooth_seg(segmentation, pad_prev=pad_prev, pad_next=pad_next)

                    # Note: the pad_next value here is only relevant if there is a leftover from last batch
                    prev_segmentation = raw_segmentation[-(r * 2 - pad_next):]

                mask = segmentation.argmax(1) == 1

                # Encode segmentation
                for b in range(mask.shape[0]):
                    curr_mask = mask[b].cpu().numpy()
                    if self.seg_remove_mouth:
                        curr_mask = remove_inner_mouth(curr_mask, landmarks[frame_count])
                        frame_count += 1
                    encoded_segmentations.append(encode_binary_mask(curr_mask))

            # Final iteration if we have leftover unsmoothed segmentations from the last batch
            if pad_next < r:
                # Compute segmentation
                segmentation = self.smooth_seg(prev_segmentation, pad_prev=pad_prev, pad_next=r)
                mask = segmentation.argmax(1) == 1

                # Encode segmentation
                for b in range(mask.shape[0]):
                    curr_mask = mask[b].cpu().numpy()
                    if self.seg_remove_mouth:
                        curr_mask = remove_inner_mouth(curr_mask, landmarks[frame_count])
                        frame_count += 1
                    encoded_segmentations.append(encode_binary_mask(curr_mask))

            # Write to file
            with open(curr_seg_path, "wb") as fp:  # Pickling
                pickle.dump(encoded_segmentations, fp)


    def cache(self, input_path, output_dir=None):
        # Validation
        assert os.path.isfile(input_path), 'Input path "%s" does not exist' % input_path
        assert output_dir is None or os.path.isdir(output_dir), 'Output path "%s" must be a directory' % output_dir
        is_vid = os.path.splitext(input_path)[1] == '.mp4'

        # Set paths
        output_dir = os.path.splitext(input_path)[0] if output_dir is None else output_dir
        det_file_path = os.path.splitext(input_path)[0] + self.det_postfix
        if not os.path.isfile(det_file_path):   # Check if there is a detection file in the same directory as the video
            det_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(input_path))[0] +
                                         self.det_postfix)
        seq_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(input_path))[0] + self.seq_postfix)
        first_cropped_path = os.path.join(output_dir, os.path.splitext(os.path.basename(input_path))[0] +
                                          '_seq00' + os.path.splitext(input_path)[1])
        pose_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(input_path))[0] + self.pose_postfix)

        # Create directory
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

        # Face detection
        if not os.path.isfile(det_file_path):
            self.face_detector(input_path, det_file_path)

        # Detections to sequences
        if not os.path.isfile(seq_file_path):
            detections2sequences_main(input_path, seq_file_path, det_file_path, self.iou_thresh, self.min_length,
                                      self.min_size, self.crop_scale, self.center_kernel, self.size_kernel,
                                      self.smooth_det, self.display, self.write_empty)

        # Crop video sequences
        if not os.path.isfile(first_cropped_path):
            if is_vid:
                crop_video_sequences_main(input_path, output_dir, seq_file_path, self.seq_postfix, self.resolution,
                                          self.crop_scale, select='all', disable_tqdm=False)
            else:
                crop_image_sequences_main(input_path, output_dir, seq_file_path, self.seq_postfix, '.jpg',
                                          self.resolution, self.crop_scale)

        # Face poses
        # if not os.path.isfile(pose_file_path) and is_vid:
        #     self.process_pose(input_path, output_dir, seq_file_path, pose_file_path)
        # if is_vid:
        self.process_pose(input_path, output_dir, seq_file_path)

        # Extract frontal images
        # if self.cache_pose and self.cache_frontal and is_vid:
        #     self.extract_frontal_images(input_path, output_dir, pose_file_path)
        if self.cache_pose and self.cache_frontal and is_vid:
            self.extract_frontal_images(input_path, output_dir, seq_file_path)

        # Cache landmarks
        self.process_landmarks(input_path, output_dir, seq_file_path)

        # Cache segmentation
        self.process_segmentation(input_path, output_dir, seq_file_path)

        return output_dir, seq_file_path, pose_file_path if self.cache_pose and is_vid else None


class VideoProcessCallable(VideoProcessBase):
    def __init__(self, *args, **kwargs):
        super(VideoProcessCallable, self).__init__(*args, **kwargs)

    def __call__(self, input_path, output_dir=None):
        return self.cache(input_path, output_dir)


def smooth_poses(poses, kernel_size=5):
    out_poses = poses.copy() if isinstance(poses, np.ndarray) else np.array(poses)

    # Prepare smoothing kernel
    # w = np.hamming(kernel_size)
    w = np.ones(kernel_size)
    w /= w.sum()

    # Smooth poses
    poses_padded = np.pad(out_poses, ((kernel_size // 2, kernel_size // 2), (0, 0)), 'reflect')
    for i in range(out_poses.shape[1]):
        out_poses[:, i] = np.convolve(w, poses_padded[:, i], mode='valid')

    return out_poses


def main(input, output=d('output'), resolution=d('resolution'), crop_scale=d('crop_scale'), gpus=d('gpus'),
         cpu_only=d('cpu_only'), display=d('display'), verbose=d('verbose'),
         # Detection arguments:
         detection_model=d('detection_model'), det_batch_size=d('det_batch_size'), det_postfix=d('det_postfix'),
         # Sequence arguments:
         iou_thresh=d('iou_thresh'), min_length=d('min_length'), min_size=d('min_size'),
         center_kernel=d('center_kernel'), size_kernel=d('size_kernel'), smooth_det=d('smooth_det'),
         seq_postfix=d('seq_postfix'), write_empty=d('write_empty'),
         # Pose arguments:
         pose_model=d('pose_model'), pose_batch_size=d('pose_batch_size'), pose_postfix=d('pose_postfix'),
         cache_pose=d('cache_pose'), cache_frontal=d('cache_frontal'), smooth_poses=d('smooth_poses'),
         # Landmarks arguments:
         lms_model=d('lms_model'), lms_batch_size=d('lms_batch_size'), landmarks_postfix=d('landmarks_postfix'),
         cache_landmarks=d('cache_landmarks'), smooth_landmarks=d('smooth_landmarks'),
         # Segmentation arguments:
         seg_model=d('seg_model'), seg_batch_size=d('seg_batch_size'), segmentation_postfix=d('segmentation_postfix'),
         cache_segmentation=d('cache_segmentation'), smooth_segmentation=d('smooth_segmentation'),
         seg_remove_mouth=d('seg_remove_mouth')):
    video_process = VideoProcessCallable(
        resolution, crop_scale, gpus, cpu_only, display, verbose,
        detection_model=detection_model, det_batch_size=det_batch_size, det_postfix=det_postfix,
        iou_thresh=iou_thresh, min_length=min_length, min_size=min_size, center_kernel=center_kernel,
        size_kernel=size_kernel, smooth_det=smooth_det, seq_postfix=seq_postfix, write_empty=write_empty,
        pose_model=pose_model, pose_batch_size=pose_batch_size, pose_postfix=pose_postfix, cache_pose=cache_pose,
        cache_frontal=cache_frontal, smooth_poses=smooth_poses, lms_model=lms_model, lms_batch_size=lms_batch_size,
        landmarks_postfix=landmarks_postfix, cache_landmarks=cache_landmarks, smooth_landmarks=smooth_landmarks,
        seg_model=seg_model, seg_batch_size=seg_batch_size, segmentation_postfix=segmentation_postfix,
        cache_segmentation=cache_segmentation, smooth_segmentation=smooth_segmentation,
        seg_remove_mouth=seg_remove_mouth)
    if len(input) == 1 and os.path.isfile(input[0]):
        video_process.cache(input, output)
    else:
        batch(input, None, output, video_process, postfix='.mp4')


if __name__ == "__main__":
    main(**vars(parser.parse_args()))



In [0]:
#@title 3，使用 mp4v 需要修改 fsgan/preprocess/crop_image_sequences.py
%%writefile fsgan/preprocess/crop_image_sequences.py
import os
import pickle
from tqdm import tqdm
import numpy as np
import cv2
from fsgan.utils.bbox_utils import scale_bbox, crop_img
from fsgan.utils.video_utils import Sequence


def main(input_path, output_dir=None, cache_path=None, seq_postfix='_dsfd_seq.pkl', resolution=256, crop_scale=2.0,
         select='all', disable_tqdm=False):
    cache_path = os.path.splitext(input_path)[0] + seq_postfix if cache_path is None else cache_path
    if output_dir is None:
        output_dir = os.path.splitext(input_path)[0]
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

    # Verification
    if not os.path.isfile(input_path):
        raise RuntimeError('Input video does not exist: ' + input_path)
    if not os.path.isfile(cache_path):
        raise RuntimeError('Cache file does not exist: ' + cache_path)
    if not os.path.isdir(output_dir):
        raise RuntimeError('Output directory does not exist: ' + output_dir)

    print('=> Cropping video sequences from video: "%s"...' % os.path.basename(input_path))

    # Load sequences from file
    with open(cache_path, "rb") as fp:  # Unpickling
        seq_list = pickle.load(fp)

    # Select sequences
    if select == 'longest':
        selected_seq_index = np.argmax([len(s) for s in seq_list])
        seq = seq_list[selected_seq_index]
        seq.id = 0
        seq_list = [seq]

    # Open input video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise RuntimeError('Failed to read video: ' + input_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    input_vid_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    input_vid_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # For each sequence initialize output video file
    out_vids = []
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    for seq in seq_list:
        curr_vid_name = os.path.splitext(os.path.basename(input_path))[0] + '_seq%02d.mp4' % seq.id
        curr_vid_path = os.path.join(output_dir, curr_vid_name)
        out_vids.append(cv2.VideoWriter(curr_vid_path, fourcc, fps, (resolution, resolution)))

    # For each frame in the target video
    cropped_detections = [[] for seq in seq_list]
    cropped_landmarks = [[] for seq in seq_list]
    pbar = range(total_frames) if disable_tqdm else tqdm(range(total_frames))
    for i in pbar:
        ret, frame = cap.read()
        if frame is None:
            continue

        # For each sequence
        for s, seq in enumerate(seq_list):
            if i < seq.start_index or (seq.start_index + len(seq) - 1) < i:
                continue
            det = seq[i - seq.start_index]

            # Crop frame
            bbox = np.concatenate((det[:2], det[2:] - det[:2]))
            bbox = scale_bbox(bbox, crop_scale)
            frame_cropped = crop_img(frame, bbox)
            frame_cropped = cv2.resize(frame_cropped, (resolution, resolution), interpolation=cv2.INTER_CUBIC)

            # Write cropped frame to output video
            out_vids[s].write(frame_cropped)

            # Add cropped detection to list
            orig_size = bbox[2:]
            axes_scale = np.array([resolution, resolution]) / orig_size
            det[:2] -= bbox[:2]
            det[2:] -= bbox[:2]
            det[:2] *= axes_scale
            det[2:] *= axes_scale
            cropped_detections[s].append(det)

            # Add cropped landmarks to list
            if hasattr(seq, 'landmarks'):
                curr_landmarks = seq.landmarks[i - seq.start_index]
                curr_landmarks[:, :2] -= bbox[:2]

                # 3D landmarks case
                if curr_landmarks.shape[1] == 3:
                    axes_scale = np.append(axes_scale, axes_scale.mean())

                curr_landmarks *= axes_scale
                cropped_landmarks[s].append(curr_landmarks)

    # For each sequence write cropped sequence to file
    for s, seq in enumerate(seq_list):
        # seq.detections = np.array(cropped_detections[s])
        # if hasattr(seq, 'landmarks'):
        #     seq.landmarks = np.array(cropped_landmarks[s])
        # seq.start_index = 0

        # TODO: this is a hack to change class type (remove this later)
        out_seq = Sequence(0)
        out_seq.detections = np.array(cropped_detections[s])
        if hasattr(seq, 'landmarks'):
            out_seq.landmarks = np.array(cropped_landmarks[s])
        out_seq.id, out_seq.obj_id, out_seq.size_avg = seq.id, seq.obj_id, seq.size_avg

        # Write to file
        curr_out_name = os.path.splitext(os.path.basename(input_path))[0] + '_seq%02d%s' % (out_seq.id, seq_postfix)
        curr_out_path = os.path.join(output_dir, curr_out_name)
        with open(curr_out_path, "wb") as fp:  # Pickling
            pickle.dump([out_seq], fp)


if __name__ == "__main__":
    # Parse program arguments
    import argparse
    parser = argparse.ArgumentParser('crop_video_sequences')
    parser.add_argument('input', metavar='VIDEO',
                        help='path to input video')
    parser.add_argument('-o', '--output', metavar='DIR',
                        help='output directory')
    parser.add_argument('-c', '--cache', metavar='PATH',
                        help='path to sequence cache file')
    parser.add_argument('-sp', '--seq_postfix', default='_dsfd_seq.pkl', metavar='POSTFIX',
                        help='input sequence file postfix')
    parser.add_argument('-r', '--resolution', default=256, type=int, metavar='N',
                        help='output video resolution (default: 256)')
    parser.add_argument('-cs', '--crop_scale', default=2.0, type=float, metavar='F',
                        help='crop scale relative to bounding box (default: 2.0)')
    parser.add_argument('-s', '--select', default='all', metavar='STR',
                        help='selection method [all|longest]')
    parser.add_argument('-dt', '--disable_tqdm', dest='disable_tqdm', action='store_true',
                          help='if specified disables tqdm progress bar')
    args = parser.parse_args()
    main(args.input, args.output, args.cache, args.seq_postfix, args.resolution, args.crop_scale, args.select,
         args.disable_tqdm)


# 使用

python swap.py SOURCE -t TARGET -o OUTPUT [FLAGS]

SOURCE 为源 mp4

TARGET 为目标 mp4

OUTPUT 输出路径

[FLAGS] 各类参数

In [0]:
# 1，生成最佳质量的视频
%cd /content/drive/My Drive/fsganLab/fshome
!python swap.py '/content/drive/My Drive/fsganLab/fshome/fsgan/docs/examples/shinzo_abe.mp4' -t '/content/drive/My Drive/fsganLab/fshome/fsgan/docs/examples/girl-B.mp4' -o . --finetune --finetune_save --seg_remove_mouth

/content/drive/My Drive/fsganLab/fshome
=> using GPU devices: 0
=> Loading face pose model: "hopenet_robust_alpha1.pth"...
=> Loading face landmarks model: "hr18_wflw_landmarks.pth"...
=> Loading face segmentation model: "celeba_unet_256_1_2_segmentation_v2.pth"...
=> Loading face reenactment model: "nfv_msrunet_256_1_2_reenactment_v2.1.pth"...
=> Loading face completion model: "ijbc_msrunet_256_1_2_inpainting_v2.pth"...
=> Loading face blending model: "ijbc_msrunet_256_1_2_blending_v2.pth"...
=> Detecting faces in video: "girl-B.mp4..."
100% 207/207 [00:39<00:00,  5.19frames/s]
=> Extracting sequences from detections in video: "girl-B.mp4"...
100% 208/208 [00:00<00:00, 14431.06it/s]
=> Cropping video sequences from video: "girl-B.mp4"...
100% 207/207 [00:01<00:00, 185.43it/s]
=> Computing face poses for video: "girl-B_seq00.mp4"...
100% 2/2 [00:01<00:00,  1.68batches/s]
=> Computing face landmarks for video: "girl-B_seq00.mp4"...
100% 4/4 [00:01<00:00,  2.86batches/s]
=> Computing fac

In [0]:
# 2，生成最佳效率的视频
%cd /content/drive/My Drive/fsganLab/fshome
!python swap.py ../docs/examples/shinzo_abe.mp4 -t ../docs/examples/conan_obrien.mp4 -o . --seg_remove_mouth

In [0]:
# 3，生成头部转后直方视频
%cd /content/drive/My Drive/fsganLab/fshome
!python swap.py ../docs/examples/shinzo_abe.mp4 -t ../docs/examples/conan_obrien.mp4 -o . --output_crop -f -fs -srm

In [0]:
# 4，生成比较用的直方视频
%cd /content/drive/My Drive/fsganLab/fshome
!python swap.py ../docs/examples/shinzo_abe.mp4 -t ../docs/examples/conan_obrien.mp4 -o . --verbose 1 -f -fs -srm

In [0]:
# 5，输出所有生产信息（包括过程，主要用于调试）
%cd /content/drive/My Drive/fsganLab/fshome
!python swap.py ../docs/examples/shinzo_abe.mp4 -t ../docs/examples/conan_obrien.mp4 -o . --verbose 2 -f -fs -srm